In [18]:
import pygad
import numpy
import time
import pandas as pd
import sys
sys.path.append('../data')

from collections import namedtuple
from functools import partial
from random import choices, randint, randrange, random
from typing import List, NamedTuple, Optional, Callable, Tuple

In [19]:
df= pd.read_csv('../data/products.csv')

In [29]:
df

,Product,Space,Price,Quantity
0,Refrigerator A,0.751000,999.90,3
1,Cell phone,0.000009,2199.12,2
2,TV 55,0.400000,4346.99,5
3,TV 50,0.290000,3999.90,5
4,TV 42,0.200000,2999.00,5
5,Notebook A,0.003500,2499.90,4
6,Ventilator,0.496000,199.90,12
7,Microwave A,0.042400,308.66,7
8,Microwave B,0.054400,429.90,7
9,Microwave C,0.031900,299.29,9


In [23]:
space=df['Space'].to_list()

In [7]:
#form a list from the dataframe
my_list = df.values.tolist()
my_list_1 = []
#create a list from the above list by iteriting.....
for x in my_list:
  for i in range(x[3]):
    my_list_1.append(x[:3])

my_list_1

[['Refrigerator A', 0.751, 999.9],
 ['Refrigerator A', 0.751, 999.9],
 ['Refrigerator A', 0.751, 999.9],
 ['Cell phone', 8.99e-06, 2199.12],
 ['Cell phone', 8.99e-06, 2199.12],
 ['TV 55', 0.4, 4346.99],
 ['TV 55', 0.4, 4346.99],
 ['TV 55', 0.4, 4346.99],
 ['TV 55', 0.4, 4346.99],
 ['TV 55', 0.4, 4346.99],
 ['TV 50', 0.29, 3999.9],
 ['TV 50', 0.29, 3999.9],
 ['TV 50', 0.29, 3999.9],
 ['TV 50', 0.29, 3999.9],
 ['TV 50', 0.29, 3999.9],
 ['TV 42', 0.2, 2999.0],
 ['TV 42', 0.2, 2999.0],
 ['TV 42', 0.2, 2999.0],
 ['TV 42', 0.2, 2999.0],
 ['TV 42', 0.2, 2999.0],
 ['Notebook A', 0.0035, 2499.9],
 ['Notebook A', 0.0035, 2499.9],
 ['Notebook A', 0.0035, 2499.9],
 ['Notebook A', 0.0035, 2499.9],
 ['Ventilator', 0.496, 199.9],
 ['Ventilator', 0.496, 199.9],
 ['Ventilator', 0.496, 199.9],
 ['Ventilator', 0.496, 199.9],
 ['Ventilator', 0.496, 199.9],
 ['Ventilator', 0.496, 199.9],
 ['Ventilator', 0.496, 199.9],
 ['Ventilator', 0.496, 199.9],
 ['Ventilator', 0.496, 199.9],
 ['Ventilator', 0.496, 199.

In [8]:

data= pd.DataFrame(my_list_1, columns=['Product','Space','Price'])

In [12]:
def iternamedtuple(df):
    Thing=namedtuple('Thing',df.columns)
    for row in df.itertuples():
        yield Thing(*row[1:])

df_list = list(iternamedtuple(data))
df_list

[Thing(Product='Refrigerator A', Space=0.751, Price=999.9),
 Thing(Product='Refrigerator A', Space=0.751, Price=999.9),
 Thing(Product='Refrigerator A', Space=0.751, Price=999.9),
 Thing(Product='Cell phone', Space=8.99e-06, Price=2199.12),
 Thing(Product='Cell phone', Space=8.99e-06, Price=2199.12),
 Thing(Product='TV 55', Space=0.4, Price=4346.99),
 Thing(Product='TV 55', Space=0.4, Price=4346.99),
 Thing(Product='TV 55', Space=0.4, Price=4346.99),
 Thing(Product='TV 55', Space=0.4, Price=4346.99),
 Thing(Product='TV 55', Space=0.4, Price=4346.99),
 Thing(Product='TV 50', Space=0.29, Price=3999.9),
 Thing(Product='TV 50', Space=0.29, Price=3999.9),
 Thing(Product='TV 50', Space=0.29, Price=3999.9),
 Thing(Product='TV 50', Space=0.29, Price=3999.9),
 Thing(Product='TV 50', Space=0.29, Price=3999.9),
 Thing(Product='TV 42', Space=0.2, Price=2999.0),
 Thing(Product='TV 42', Space=0.2, Price=2999.0),
 Thing(Product='TV 42', Space=0.2, Price=2999.0),
 Thing(Product='TV 42', Space=0.2, Pri

In [ ]:
from random import choices, randint
from typing import List, Tuple

Genome = List[int]
things=df_list
function_inputs= things
length=len(things)
weight_limit= 2

def generate_genome(length: int) -> Genome:
    return choices([0, 1], k=length)
def on_start(ga):
    length=len(things)
    return choices([0, 1], k=length)

genome= generate_genome(length)

def fitness(solution, solution_idx):
    if len(genome) != len(things):
        raise ValueError("GENOME AND THINGS MUST BE OF SAME LENGTH")
    weight=0
    value=0
    for i, thing in enumerate(things):
        if genome [i]==1:
            weight +=thing.Space
            value += thing.Price
            if weight > weight_limit:
                return 0
    fitness=value
    return fitness
def single_point_crossover(a: Genome, b: Genome) -> Tuple[Genome, Genome]:
    if len(a) != len(b):
        raise ValueError("Genomes a and b must be of same length")

    length = len(a)
    if length < 2:
        return a, b

    p = randint(1, length - 1)
    return a[0:p] + b[p:], b[0:p] + a[p:]

num_generations = 100 # Number of generations.
num_parents_mating = 10 # Number of solutions to be selected as parents in the mating pool.

sol_per_pop = 20 # Number of solutions in the population.
num_genes = len(function_inputs)

last_fitness = 0

def on_generation(ga):
    print("Generation", ga.generations_completed)
    print(ga.population)
 

ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating,
                       sol_per_pop=sol_per_pop,
                       num_genes=num_genes,
                       fitness_func=fitness,
                       on_generation=on_generation,
                       on_start=on_start,
                       save_best_solutions=False)

# Running the GA to optimize the parameters of the function.
ga_instance.run()

ga_instance.plot_fitness()  

solution, solution_fitness, solution_idx = ga_instance.best_solution()
print("Parameters of the best solution :", solution)
print("Fitness value of the best solution :", solution_fitness, "\n")
print("Index of the best solution :", solution_idx, "\n")

In [78]:

from collections import namedtuple
from functools import partial
from random import choices, randint, randrange, random
from typing import List, NamedTuple, Optional, Callable, Tuple

Genome = List[int]
Population = List[Genome]
PopulateFunc = Callable[[], Population]
FitnessFunc = Callable[[Genome], int]
SelectionFunc = Callable[[Population, FitnessFunc], Tuple[Genome, Genome]]
CrossoverFunc = Callable[[Genome, Genome], Tuple[Genome, Genome]]
MutationFunc = Callable[[Genome], Genome]
PrinterFunc = Callable[[Population, int, FitnessFunc], None]
Thing= namedtuple('Thing',['Product','Space','Price'])

things= df_list

def generate_genome(length: int) -> Genome:
    return choices([0, 1], k=length)
def generate_population(size: int, genome_length: int) -> Population:
    return [generate_genome(genome_length) for _ in range(size)]
def fitness(genome:Genome, things:Thing, weight_limit:int) -> int:
    if len(genome) != len(things):
        raise ValueError("GENOME AND THINGS MUST BE OF SAME LENGTH")
    weight=0
    value=0
    for i, thing in enumerate(things):
        if genome [i]==1:
            weight +=thing.Space
            value += thing.Price
            if weight > weight_limit:
                return 0
    return value
def single_point_crossover(a: Genome, b: Genome) -> Tuple[Genome, Genome]:
    if len(a) != len(b):
        raise ValueError("Genomes a and b must be of same length")

    length = len(a)
    if length < 2:
        return a, b

    p = randint(1, length - 1)
    return a[0:p] + b[p:], b[0:p] + a[p:]


def mutation(genome: Genome, num: int = 1, probability: float = 0.5) -> Genome:
    for _ in range(num):
        index = randrange(len(genome))
        genome[index] = genome[index] if random() > probability else abs(genome[index] - 1)
    return genome


def population_fitness(population: Population, fitness_func: FitnessFunc) -> int:
    return sum([fitness_func(genome) for genome in population])


def selection_pair(population: Population, fitness_func: FitnessFunc) -> Population:
    return choices(
        population=population,
        weights=[fitness_func(genome) for genome in population],
        k=2
    )


def sort_population(population: Population, fitness_func: FitnessFunc) -> Population:
    return sorted(population, key=fitness_func, reverse=True)


def genome_to_string(genome: Genome) -> str:
    return "".join(map(str, genome))


def print_stats(population: Population, generation_id: int, fitness_func: FitnessFunc):
    print("GENERATION %02d" % generation_id)
    print("=============")
    print("Population: [%s]" % ", ".join([genome_to_string(gene) for gene in population]))
    print("Avg. Fitness: %f" % (population_fitness(population, fitness_func) / len(population)))
    sorted_population = sort_population(population, fitness_func)
    print(
        "Best: %s (%f)" % (genome_to_string(sorted_population[0]), fitness_func(sorted_population[0])))
    print("Worst: %s (%f)" % (genome_to_string(sorted_population[-1]),
                              fitness_func(sorted_population[-1])))
    print("")

    return sorted_population[0]

def run_evolution(
        populate_func: PopulateFunc,
        fitness_func: FitnessFunc,
        fitness_limit: int,
        selection_func: SelectionFunc = selection_pair,
        crossover_func: CrossoverFunc = single_point_crossover,
        mutation_func: MutationFunc = mutation,
        generation_limit: int = 100,
        printer: Optional[PrinterFunc] = None) \
        -> Tuple[Population, int]:
    population = populate_func()

    for i in range(generation_limit):
        population = sorted(population, key=lambda genome: fitness_func(genome), reverse=True)

        if printer is not None:
            printer(population, i, fitness_func)

        if fitness_func(population[0]) >= fitness_limit:
            break

        next_generation = population[0:2]

        for j in range(int(len(population) / 2) - 1):
            parents = selection_func(population, fitness_func)
            offspring_a, offspring_b = crossover_func(parents[0], parents[1])
            offspring_a = mutation_func(offspring_a)
            offspring_b = mutation_func(offspring_b)
            next_generation += [offspring_a, offspring_b]

        population = next_generation
    population=sorted(
        population,
        key=lambda genome:fitness_func(genome),
        reverse=True
    )

    return population, i 
population, generations=run_evolution(
    populate_func=partial(generate_population, size=10, genome_length=len(things)),
    fitness_func=partial(fitness, things=things, weight_limit=4),
    fitness_limit=1000,
    generation_limit=10
 
)


def genome_to_things (genome: Genome, things: [Thing])->[Thing]:
    result =[]
    for i,thing in enumerate(things):
        if genome[i]==1:
            result+= [thing.Product]
    return result

print (f"Number of generations: {generations}")
print(f"best solution:{genome_to_things(population[0], things)}")
print(f"")

Number of generations: 9
best solution:['Refrigerator A', 'Refrigerator A', 'TV 55', 'TV 55', 'TV 55', 'TV 50', 'TV 50', 'TV 50', 'TV 42', 'TV 42', 'Notebook A', 'Notebook A', 'Ventilator', 'Ventilator', 'Ventilator', 'Ventilator', 'Ventilator', 'Ventilator', 'Ventilator', 'Microwave A', 'Microwave A', 'Microwave A', 'Microwave A', 'Microwave A', 'Microwave A', 'Microwave B', 'Microwave C', 'Microwave C', 'Microwave C', 'Microwave C', 'Refrigerator B', 'Refrigerator B', 'Refrigerator C', 'Refrigerator C', 'Notebook B', 'Notebook B', 'Notebook C']

